In [1]:
# ref
# https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html

In [11]:
# PROXY
import sys
sys.path.append('.')
#import fox_proxy


import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.8.0a0+6e6231f


In [12]:
# BERT Constant
_CLS = '[CLS]'
_SEP = '[SEP]'
_UNK = '[UNK]'
_PAD = '[PAD]'
_MASK = '[MASK]'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [13]:
# dictionary in tokenizer
vocab = tokenizer.vocab
print('字典大小: ', len(vocab))


字典大小:  21128


In [14]:
# 瞧瞧中文 BERT 字典裡頭紀錄的一些 tokens 以及其對應的索引
import random
random_tokens = random.sample(list(vocab),10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format('token', 'index'))
print("-" * 25)
for t, id in zip(random_tokens, random_ids):
    print("{0:20}{1:15}".format(t, id))

token               index          
-------------------------
##戍                           15822
##梢                           16513
##＊                           21079
276                           11173
cms                           11539
铲                              7211
##渴                           17008
##った                           9190
てすか                           11244
問                              1558


In [15]:
#  ㄅㄆㄇㄈ
indices = list(range(647,657))
some_pairs = [(t,idx) for t, idx in vocab.items() if idx in indices ]
for pair in some_pairs:
    print(pair)


('ㄅ', 647)
('ㄆ', 648)
('ㄇ', 649)
('ㄉ', 650)
('ㄋ', 651)
('ㄌ', 652)
('ㄍ', 653)
('ㄎ', 654)
('ㄏ', 655)
('ㄒ', 656)


In [16]:
# 讓我們利用中文 BERT 的 tokenizer 將一個中文句子斷詞看看：
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(text)
print(tokens[:10],'...')
print(ids[:10], '...')



[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761] ...


In [17]:
"""
這段程式碼載入已經訓練好的 masked 語言模型並對有 [MASK] 的句子做預測
"""
from transformers import BertForMaskedLM

# 除了tokens 以外我們還需要辨別句子的segment ids
tokens_tensor = torch.tensor([ids]) #(1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor) #(1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
# clear_output()

# 使用  masked LM 估計 [MASK] 位置所代表的實際 token
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
    #(1, seq_len, num_hidden_units)

del maskedLM_model

# 將 [MASK] 位置的機率分布取 top k 最有可能的 tokens 出來
masked_index = 5
k = 3

probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

#顯示 top k 可能的字 , 一般我們取top 1 當預測值
print('輸入 tokens : ', tokens[:10], '...' )
print('-' * 50)
for i,(t, p) in enumerate(zip(predicted_tokens, probs),1):
    tokens[masked_index] = t
    print('Top {} ({:2}%):{}'.format(i,int(p.item() * 100), tokens[:10]),'...')
          
    

輸入 tokens :  ['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
--------------------------------------------------
Top 1 (67%):['[CLS]', '等', '到', '潮', '水', '來', '了', '，', '就', '知'] ...
Top 2 (25%):['[CLS]', '等', '到', '潮', '水', '濕', '了', '，', '就', '知'] ...
Top 3 ( 2%):['[CLS]', '等', '到', '潮', '水', '過', '了', '，', '就', '知'] ...


In [18]:
# BERT 的視覺化工具 BertViz
# import sys
# !test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
# if not 'bertviz_repo' in sys.path:
#   sys.path += ['bertviz_repo']
sys.path.append('./bertviz-master')
# import packages
from transformers import BertTokenizer, BertModel
from bertviz import head_view

# 在 jupyter notebook 裡頭顯示 visualzation 的 helper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

clear_output()

In [22]:
# visualize
model_version = 'bert-base-chinese'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)

# 情境1
sentence_a = '胖虎叫大雄去買漫畫，'
sentence_b = '回來慢了就打他。'

# 得到tokens 後丟入 bert 取得 attention
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_sprcial_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()
head_view(attention, tokens)


<IPython.core.display.Javascript object>

# ---

# test txt classify

# ---

    1.準備原始文本數據
    2.將原始文本轉換成 BERT 相容的輸入格式
    3.在 BERT 之上加入新 layer 成下游任務模型
    4.訓練該下游任務模型
    5.對新樣本做推論


In [23]:
# https://www.kaggle.com/c/fake-news-pair-classification-challenge/data
# =============================1. prepare original txt data
import glob
glob.glob("*.csv.zip")

['test.csv.zip', 'train.csv.zip']

In [24]:
"""
前處理原始的訓練數據集。
你不需了解細節，只需要看註解了解邏輯或是輸出的數據格式即可
"""
import os
import pandas as pd

#unzip train file
#os.system('unzip train.csv.zip')

# data process remove empty title examples
df_train = pd.read_csv('train.csv')
empty_title = ((df_train['title2_zh'].isnull()) 
              | (df_train['title1_zh'].isnull())  
              | (df_train['title2_zh'] == '') 
              | (df_train['title2_zh'] == '0'))
df_train = df_train[~empty_title]

# 剔除過長樣本 避免 BERT 無法將整個輸入序列 放入記憶體不多的 CPU

MAX_LENGTH = 30
df_train = df_train[~empty_title]

#剔除 過長樣本 避免 bert 無法將整個輸入序列 放入記憶體不多的gpu
MAX_LENGTH  = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x: len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x: len(x)) > MAX_LENGTH)]

# 只用 1% 的訓練數據 看看 BERT 對少量標註數據有多少幫助
SAMPLE_FRAC = 0.01
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh','title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

# idempotence 冪等 , 將處理結果 另存成 csv 供 Pytorch 使用
df_train.to_csv('train2.csv', encoding='utf_8_sig', index=False)
print('訓練樣本數:', len(df_train))
df_train.head()

訓練樣本數: 2657


<ipython-input-24-25f95bcfc0c2>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train = df_train[~empty_title]


,text_a,text_b,label
0,苏有朋要结婚了，但网友觉得他还是和林心如比较合适,好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！,unrelated
1,爆料李小璐要成前妻了贾乃亮模仿王宝强一步到位、快刀斩乱麻！,李小璐要变前妻了？贾乃亮可能效仿王宝强当机立断，快刀斩乱麻！,agreed
2,为彩礼，母亲把女儿嫁给陌生男子，十年后再见面，母亲湿了眼眶,阿姨，不要彩礼是觉得你家穷，给你台阶下，不要以为我嫁不出去！,unrelated
3,猪油是个宝，一勺猪油等于十副药，先备起来再说,传承千百的猪油为何变得人人唯恐避之不及？揭开猪油的四大谣言！,unrelated
4,剖析：香椿，为什么会致癌？,香椿含亚硝酸盐多吃会致癌？测完发现是谣言,disagreed


In [25]:
df_train.label.value_counts() / len(df_train)


unrelated    0.679338
agreed       0.294317
disagreed    0.026346
Name: label, dtype: float64

In [26]:
df_test = pd.read_csv('test.csv')
df_test = df_test.loc[:, ['title1_zh', 'title2_zh', 'id']]
df_test.columns = ['text_a', 'text_b', 'id']
df_test.to_csv('test2.csv',  encoding='utf_8_sig', index=False)
print('預測樣本數: ', len(df_test))
df_test.head()


預測樣本數:  80126


,text_a,text_b,id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [27]:
ratio = len(df_test) / len(df_train)
print('測試集樣本數 / 訓練集樣本數 = {:.1f}倍'.format(ratio))


測試集樣本數 / 訓練集樣本數 = 30.2倍


In [28]:
# 2. 將文本轉成 BBRT 相容的輸入格式
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
import pysnooper

# class FakeNewsDataset(Dataset):
#     # 讀取之前處理後的csv 並初始化參數
#     def __init__(self, mode, tokenizer):
#         assert mode in ['train', 'test']
#         self.mode = mode
#         #大數據你會需要用 iterator = True
#         self.df = pd.read_csv(mode + '2.csv', encoding='utf_8_sig')
#         self.len = len(self.df)
#         self.label_map = {'agreed': 0, 'disagreed':1, 'unrelated': 2}
#         self.tokenizer = tokenizer # Bert tokenizer
    
#     # 回傳一筆訓練 / 測試數據
   
#     def __getitem__(self, idx):
#         if self.mode =='test':
#             text_a, text_b = self.df.iloc[idx, :2].values
#             label_tensor = None
#         else:
#             text_a, text_b, label = self.df.iloc[idx,:].values
#             # 將label 文字 轉換成索引方便的tensor
#             label_id = self.label_map[label]
#             label_tensor = torch.tensor(label_id)
        
#         #建立第一個句子的BERT tokens 並加入分隔符號[SEP]
#         word_pieces = [_CLS]
#         token_a = self.tokenizer.tokenize(text_a)
#         word_pieces += token_a + [_SEP]
#         len_a = len(word_pieces) 
        
#         # 第二個句子 BERT tokens
#         tokens_b = self.tokenizer.tokenize(text_b)
#         word_pieces += tokens_b + [_SEP]
#         len_b = len(word_pieces) - len_a 
        
#         # 將整個 token 序列 轉成 索引 序列
#         ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
#         tokens_tensor = torch.tensor(ids)
        
#         # 將第一句話包含 [SEP]的 token位置設為0, 其他為1 表示第二句
#         segments_tensor= torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
        
#         return (tokens_tensor, segments_tensor, label_tensor)
    
#     def __len__(self):
#         return self.len
    
# # 初始化一個專門讀取訓練樣本的 Dataset 使用中文 BERT斷詞
# trainset = FakeNewsDataset('train',tokenizer=tokenizer)
 
    
class FakeNewsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + "2.csv", encoding='utf_8_sig').fillna("")
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 第二個句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = FakeNewsDataset("train", tokenizer=tokenizer)        

In [29]:
# test dataset 
# 第一個樣本
sample_idx = 0

#將原始樣本拿出比較
text_a, text_b, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的Dataset 取出轉換的 id_tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

#將 token 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = ''.join(tokens)

print(f"""[原始文本]
句子1:{text_a}
句子2:{text_b}
分類:{label}
-------------------
[Dataset 回傳的 tensors]
token_tensor: {segments_tensor}

segments_tensor:{segments_tensor}

label_tensor   :{label_tensor}

----------------------------

[還原 token_tensors]
{combined_text}
""")


[原始文本]
句子1:苏有朋要结婚了，但网友觉得他还是和林心如比较合适
句子2:好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！
分類:unrelated
-------------------
[Dataset 回傳的 tensors]
token_tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])

segments_tensor:tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor   :2

----------------------------

[還原 token_tensors]
[CLS]苏有朋要结婚了，但网友觉得他还是和林心如比较合适[SEP]好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！[SEP]



In [30]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [31]:
# test
data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data


print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([32, 63]) 
tensor([[ 101, 5722, 3300,  ...,    0,    0,    0],
        [ 101, 4255, 3160,  ..., 8013,  102,    0],
        [ 101,  711, 2506,  ..., 8013,  102,    0],
        ...,
        [ 101, 8164, 2259,  ...,    0,    0,    0],
        [ 101, 1912, 3215,  ..., 4685, 1398,  102],
        [ 101, 1290,  711,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([32, 63])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([32, 63])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

In [32]:
# 3. 在 BERT 之上加入新 layer 成下游任務模型
# 載入一個可以做中文多分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=3, bias=True)


In [33]:
from pytorch_transformers.modeling_bert import BertPreTrainedModel
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels=2):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)  # 載入預訓練 BERT
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # 簡單 linear 層
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        # BERT 輸入就是 tokens, segments, masks
        outputs = self.bert(input_ids, token_type_ids, attention_mask, ...)
        ...
        pooled_output = self.dropout(pooled_output)
        # 線性分類器將 dropout 後的 BERT repr. 轉成類別 logits
        logits = self.classifier(pooled_output)

        # 輸入有 labels 的話直接計算 Cross Entropy 回傳，方便！
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        # 有要求回傳注意矩陣的話回傳
        elif self.output_attentions:
            return all_attentions, logits
        # 回傳各類別的 logits
        return logits

ModuleNotFoundError: No module named 'pytorch_transformers'

In [34]:
model.config


{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 3,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 21128
}

In [35]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cpu
classification acc: 0.6541211893112533


In [37]:
# 算算整個分類模型以及裡頭的簡單分類器有多少參數：
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102269955
線性分類器的參數量：2307



In [38]:
# # 訓練該下游任務模型


In [39]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 6  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    

[epoch 1] loss: 47.751, acc: 0.855
[epoch 2] loss: 33.573, acc: 0.890
[epoch 3] loss: 24.550, acc: 0.929
[epoch 4] loss: 16.336, acc: 0.957
[epoch 5] loss: 13.395, acc: 0.962
[epoch 6] loss: 10.893, acc: 0.951
CPU times: user 1h 1min, sys: 17min 42s, total: 1h 18min 43s
Wall time: 31min 57s


In [ ]:
# 5. 對新樣本做推論


In [ ]:
%%time
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = FakeNewsDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=64, 
                        collate_fn=create_mini_batch)



In [ ]:
%%time
# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"Category": predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["Id"]], 
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
df_pred.head()